In [9]:
import math
import tensorflow as tf
import numpy as np
import matplotlib
import os
from tensorflow.contrib import rnn

tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/taxi_data/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)
xy_with_noise = np.genfromtxt('/Users/yeseo/Desktop/taxi_data/2015eliminated_1.txt',delimiter = ',',dtype = None)

#data_preprocessing
xy= xy[:,:27]
a = xy[:,:2]
b = xy[:,2:]
b= MinMaxScaler(b)
xy = np.hstack((a,b))

xy_with_noise = xy_with_noise[:,:27]
a_with_noise = xy_with_noise[:,:2]
b_with_noise = xy_with_noise[:,2:]
b_with_noise = MinMaxScaler(b_with_noise)
xy_with_noise = np.hstack((a_with_noise,b_with_noise))


#parameters
seq_length =48
data_dim =27
hidden_dim = 25
output_dim = 25
learning_rate = 0.01
iterations = 1500

train_size = int(len(xy)*0.7)
validation_size = int(len(xy)*0.2)

#divide data set to train,validation and test set
train_set = xy[:train_size]
validation_set = xy[train_size:train_size+validation_size]
test_set = xy[train_size+validation_size:]

train_set_with_noise = xy_with_noise[:train_size]
validation_set_with_noise = xy_with_noise[train_size:train_size+validation_size]
test_set_with_noise = xy_with_noise[train_size+validation_size:]

# build data set for rnn
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + 24, :]
        _y = time_series[i+24:i+48,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
validationX, validationY = build_dataset(validation_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

trainX_with_noise, trainY_with_noise = build_dataset(train_set_with_noise,seq_length)
validationX_with_noise, validationY_with_noise = build_dataset(validation_set_with_noise,seq_length)
testX_with_noise,testY_with_noise = build_dataset(test_set_with_noise, seq_length)


X1 = tf.placeholder(tf.float32,[None, (seq_length/2),data_dim])
Y1 = tf.placeholder(tf.float32,[None, (seq_length/2),output_dim])

X2 = tf.placeholder(tf.float32,[None, (seq_length/2),data_dim])
Y2 = tf.placeholder(tf.float32,[None, (seq_length/2),output_dim])

#LSTM CELL만들기

with tf.variable_scope("rnn1"):
    cell1 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs1,_states1 = tf.nn.dynamic_rnn(cell1,X1,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs1[:,:], output_dim,activation_fn = None)

    loss1 =tf.reduce_mean(tf.square(Y_pred-Y1))
    train1 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss1)

with tf.variable_scope("rnn2"):
    cell2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs2,_states2 = tf.nn.dynamic_rnn(cell2, X2, dtype = tf.float32)
    Y_pred_with_noise = tf.contrib.layers.fully_connected(outputs2[:,:], output_dim,activation_fn = None)

    loss2 =tf.reduce_mean(tf.square(Y_pred_with_noise-Y2))
    train2 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss2)


#RMSE 측정
targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
x2 = x1+0.3
x3 = x2+0.3
loss_for_graph = np.zeros(iterations)
x4 = np.array(range(0,iterations))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss1 = sess.run([train1,loss1],feed_dict={X1:trainX, Y1:trainY})
        print("[step: {}] loss: {}".format(i,step_loss1))
        loss_for_graph[i] = step_loss1
        _, step_loss2 = sess.run([train2,loss2],feed_dict={X2:trainX_with_noise, Y2:trainY_with_noise})
        print("[step: {}] loss: {}".format(i,step_loss2))
        
    test_predict = sess.run(Y_pred, feed_dict = {X1:validationX})
    test_predict_with_noise = sess.run(Y_pred_with_noise, feed_dict = {X2:validationX_with_noise})

    
    rmse_val = sess.run(rmse, feed_dict={targets: validationY,predictions: test_predict})
    print("RMSE: {}".format(rmse_val))
   # print("pred: {}".format(test_predict[-1,:]))
    #print("real: {}".format(testY[-1,:]))
    #print("noise: {}".format(eliminate_noise_pred[-1,:]))
    
#    plt.bar(x1,test_predict[-1,:],label = 'predict',color ='b',width = 0.1)
  #  plt.bar(x2,testY[-1,:],label = 'real',color ='g',width = 0.1)
    #plt.bar(x3,eliminate_noise_pred[-1,:],label = 'noise',color ='g',width = 0.1)
    plt.plot(x4,loss_for_graph)
    #plt.legend()
    plt.show()



[step: 0] loss: 469987.0625
[step: 0] loss: 0.3875851035118103
[step: 1] loss: 316266.90625
[step: 1] loss: 0.26090964674949646
[step: 2] loss: 225293.53125
[step: 2] loss: 0.17596584558486938
[step: 3] loss: 170295.875
[step: 3] loss: 0.11450070142745972
[step: 4] loss: 131991.21875
[step: 4] loss: 0.07579457014799118
[step: 5] loss: 102887.40625
[step: 5] loss: 0.05900811776518822
[step: 6] loss: 86077.28125
[step: 6] loss: 0.052881937474012375
[step: 7] loss: 76386.359375
[step: 7] loss: 0.049006182700395584
[step: 8] loss: 69736.484375
[step: 8] loss: 0.0450737401843071
[step: 9] loss: 64988.41796875
[step: 9] loss: 0.04124637693166733
[step: 10] loss: 62862.7109375
[step: 10] loss: 0.03778216242790222
[step: 11] loss: 62969.17578125
[step: 11] loss: 0.03484201431274414
[step: 12] loss: 63200.4765625
[step: 12] loss: 0.03269193321466446
[step: 13] loss: 62203.609375
[step: 13] loss: 0.03143831714987755
[step: 14] loss: 60241.1640625
[step: 14] loss: 0.030806541442871094
[step: 15] 

[step: 118] loss: 0.016528910025954247
[step: 119] loss: 11509.841796875
[step: 119] loss: 0.016426198184490204
[step: 120] loss: 12172.3837890625
[step: 120] loss: 0.01639014482498169
[step: 121] loss: 11781.498046875
[step: 121] loss: 0.016248589381575584
[step: 122] loss: 11566.7861328125
[step: 122] loss: 0.01626085303723812
[step: 123] loss: 11795.318359375
[step: 123] loss: 0.016120901331305504
[step: 124] loss: 11061.666015625
[step: 124] loss: 0.016089826822280884
[step: 125] loss: 11540.5673828125
[step: 125] loss: 0.015970956534147263
[step: 126] loss: 11113.482421875
[step: 126] loss: 0.01595638506114483
[step: 127] loss: 11252.390625
[step: 127] loss: 0.01582198776304722
[step: 128] loss: 10680.20703125
[step: 128] loss: 0.015784751623868942
[step: 129] loss: 11125.0986328125
[step: 129] loss: 0.01567051000893116
[step: 130] loss: 10602.2138671875
[step: 130] loss: 0.015643516555428505
[step: 131] loss: 10601.580078125
[step: 131] loss: 0.015520959161221981
[step: 132] loss

[step: 232] loss: 0.010193495079874992
[step: 233] loss: 7065.83203125
[step: 233] loss: 0.010292790830135345
[step: 234] loss: 7034.7578125
[step: 234] loss: 0.010295883752405643
[step: 235] loss: 7013.8330078125
[step: 235] loss: 0.01005904097110033
[step: 236] loss: 7007.10595703125
[step: 236] loss: 0.01007386576384306
[step: 237] loss: 6983.6552734375
[step: 237] loss: 0.010240846313536167
[step: 238] loss: 6952.5087890625
[step: 238] loss: 0.010210087522864342
[step: 239] loss: 6933.3994140625
[step: 239] loss: 0.009963992983102798
[step: 240] loss: 6922.4775390625
[step: 240] loss: 0.01002119667828083
[step: 241] loss: 6903.357421875
[step: 241] loss: 0.010030782781541348
[step: 242] loss: 6877.48095703125
[step: 242] loss: 0.010043852962553501
[step: 243] loss: 6854.833984375
[step: 243] loss: 0.009885935112833977
[step: 244] loss: 6839.63232421875
[step: 244] loss: 0.009940887801349163
[step: 245] loss: 6827.658203125
[step: 245] loss: 0.009956274181604385
[step: 246] loss: 68

[step: 346] loss: 6882.10888671875
[step: 346] loss: 0.008985836058855057
[step: 347] loss: 7484.82763671875
[step: 347] loss: 0.008963492698967457
[step: 348] loss: 6362.30419921875
[step: 348] loss: 0.008951921947300434
[step: 349] loss: 7150.6103515625
[step: 349] loss: 0.008955421857535839
[step: 350] loss: 6299.36865234375
[step: 350] loss: 0.00895559974014759
[step: 351] loss: 7052.51953125
[step: 351] loss: 0.008952982723712921
[step: 352] loss: 6216.04833984375
[step: 352] loss: 0.008943301625549793
[step: 353] loss: 6941.9130859375
[step: 353] loss: 0.00892813503742218
[step: 354] loss: 6100.67626953125
[step: 354] loss: 0.008918296545743942
[step: 355] loss: 6774.83056640625
[step: 355] loss: 0.008910102769732475
[step: 356] loss: 6331.203125
[step: 356] loss: 0.0089093754068017
[step: 357] loss: 6686.6318359375
[step: 357] loss: 0.008905316703021526
[step: 358] loss: 5988.68212890625
[step: 358] loss: 0.008907457813620567
[step: 359] loss: 6555.365234375
[step: 359] loss: 0.

[step: 459] loss: 6045.4072265625
[step: 459] loss: 0.00866563431918621
[step: 460] loss: 7324.35498046875
[step: 460] loss: 0.008604130707681179
[step: 461] loss: 9857.91015625
[step: 461] loss: 0.008511973544955254
[step: 462] loss: 7036.3037109375
[step: 462] loss: 0.00849924422800541
[step: 463] loss: 5472.5458984375
[step: 463] loss: 0.008574300445616245
[step: 464] loss: 6793.130859375
[step: 464] loss: 0.008599601686000824
[step: 465] loss: 5678.609375
[step: 465] loss: 0.008555568754673004
[step: 466] loss: 6259.15478515625
[step: 466] loss: 0.00848702434450388
[step: 467] loss: 5797.5107421875
[step: 467] loss: 0.008483028039336205
[step: 468] loss: 5924.64697265625
[step: 468] loss: 0.008525741286575794
[step: 469] loss: 5720.7333984375
[step: 469] loss: 0.008566569536924362
[step: 470] loss: 5780.232421875
[step: 470] loss: 0.008556346409022808
[step: 471] loss: 5895.7158203125
[step: 471] loss: 0.008508148603141308
[step: 472] loss: 5670.3564453125
[step: 472] loss: 0.00846

[step: 573] loss: 4770.6455078125
[step: 573] loss: 0.008225817233324051
[step: 574] loss: 4876.1767578125
[step: 574] loss: 0.008268485777080059
[step: 575] loss: 4846.1875
[step: 575] loss: 0.008211257867515087
[step: 576] loss: 4724.85986328125
[step: 576] loss: 0.00826515443623066
[step: 577] loss: 4754.142578125
[step: 577] loss: 0.008192842826247215
[step: 578] loss: 4790.8212890625
[step: 578] loss: 0.008235974237322807
[step: 579] loss: 4743.30078125
[step: 579] loss: 0.008200755342841148
[step: 580] loss: 4753.3994140625
[step: 580] loss: 0.008198263123631477
[step: 581] loss: 4678.3134765625
[step: 581] loss: 0.00820200890302658
[step: 582] loss: 4672.43408203125
[step: 582] loss: 0.008170933462679386
[step: 583] loss: 4703.533203125
[step: 583] loss: 0.008200997486710548
[step: 584] loss: 4694.0703125
[step: 584] loss: 0.00816341582685709
[step: 585] loss: 4685.9580078125
[step: 585] loss: 0.008172468282282352
[step: 586] loss: 4703.853515625
[step: 586] loss: 0.008167649619

KeyboardInterrupt: 